In [1]:
import glob
global files

In [2]:
files = glob.glob("raw_data/*.txt")

In [3]:
files

['raw_data/2-VLSP2018-SA-Restaurant-dev (7-3-2018).txt',
 'raw_data/2-VLSP2018-SA-Hotel-dev (7-3-2018).txt',
 'raw_data/3-VLSP2018-SA-Restaurant-test (8-3-2018).txt',
 'raw_data/1-VLSP2018-SA-Restaurant-train (7-3-2018).txt',
 'raw_data/3-VLSP2018-SA-Hotel-test (8-3-2018).txt',
 'raw_data/1-VLSP2018-SA-Hotel-train (7-3-2018).txt']

In [4]:
def read_txt_file(files):
    data = []
    for file in files:
        with open(file,encoding='utf-8') as f:
            val = list(filter(None, [w.strip() for w in f.readlines()]))
            data.append(val)
    return data

In [5]:
txt_data = read_txt_file(files)

In [6]:
for data in txt_data:
    print(str("len: {}".format(len(data))))

len: 3870
len: 6000
len: 1500
len: 8883
len: 1800
len: 9000


In [7]:
def txt_to_record(txt_data):
    output = []
    for data in txt_data:
        records = []
        for i in range(0, len(data),3):
            records.append([data[i], data[i+1], data[i+2]])
        output.append(records)
    return output

In [8]:
records = txt_to_record(txt_data)

In [10]:
import pandas as pd
import re

def get_value(column_name, aspects):
    for aspect in aspects:
        if(column_name == aspect[0]):
            if(aspect[1] == 'positive'):
                return 1
            elif(aspect[1] == 'negative'):
                return -1
            else:
                return 0.5
    return -0.5
    

def fill_value(df):
    columns = df.columns[5:]
    for column in columns:
        aspect_polarity = column.split('_')
        df[column] = [get_value(column, aspects) for aspects in df['Result']]
    return df

def get_all_category_in_data_frame(targetLabels):
    categories = []
    for multiLabels in targetLabels:
        for label in multiLabels:
            if(label not in categories):
                categories.append(label)
    return categories

def records_to_data_frame(records):
    aspect = [item[2] for item in records]
    
    patern = r'\w*#\w*&*\w*\,\s*\w*'
    aspect = [re.findall(patern,item) for item in aspect]
    aspect = [[re.split(r', ',subItem) for subItem in item] for item in aspect]
    
    df = pd.DataFrame(records, columns =['Index', 'Review', 'Result'])
    df.Result = aspect
    
    df['Aspect'] = [[subItem[0] for subItem in item] for item in df.Result]
    df['Opinion'] = [[subItem[1] for subItem in item] for item in df.Result]
    
    aspect_columns = get_all_category_in_data_frame(list(df['Aspect']))
    
    dfCategories = pd.DataFrame([], columns = aspect_columns)
    df = pd.concat([df, dfCategories], axis=1, sort=False)
    df['Review'] = df['Review'].str.lower()
    df = fill_value(df)
    
    return df

In [11]:
df_list = []
for record in records:
    df_list.append(records_to_data_frame(record))

In [12]:
df_list[1].head()

,Index,Review,Result,Aspect,Opinion,FACILITIES#DESIGN&FEATURES,SERVICE#GENERAL,LOCATION#GENERAL,HOTEL#DESIGN&FEATURES,HOTEL#COMFORT,...,FACILITIES#PRICES,ROOMS#PRICES,FACILITIES#CLEANLINESS,FACILITIES#MISCELLANEOUS,DRINKS#MISCELLANEOUS,ROOMS#MISCELLANEOUS,DRINKS#PRICES,ROOMS#QUALITY,ROOM_AMENITIES#MISCELLANEOUS,ROOM_AMENITIES#PRICES
0,﻿#1,chưa có thang máy. chưa chấp nhận thanh toán b...,"[[FACILITIES#DESIGN&FEATURES, negative], [SERV...","[FACILITIES#DESIGN&FEATURES, SERVICE#GENERAL, ...","[negative, positive, positive, positive, posit...",-1.0,1.0,1.0,1.0,1.0,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
1,#2,chúng tôi không cảm thấy thoải mái vì ở chỉ 1 ...,"[[HOTEL#COMFORT, negative], [FACILITIES#COMFOR...","[HOTEL#COMFORT, FACILITIES#COMFORT, FACILITIES...","[negative, negative, negative, positive, posit...",-0.5,1.0,-0.5,-0.5,-1.0,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
2,#3,"nhân viên rất nhiệt tình thân thiện, luôn giúp...","[[SERVICE#GENERAL, positive], [ROOM_AMENITIES#...","[SERVICE#GENERAL, ROOM_AMENITIES#QUALITY, ROOM...","[positive, positive, positive, positive, posit...",-0.5,1.0,-0.5,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
3,#4,khách sạn khá dễ đến bằng phương tiên công cộn...,"[[LOCATION#GENERAL, positive], [SERVICE#GENERA...","[LOCATION#GENERAL, SERVICE#GENERAL, ROOMS#CLEA...","[positive, positive, positive, positive, posit...",-0.5,1.0,1.0,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
4,#5,"sạch sẽ, vị trí rất thuận tiện, nhân viên thân...","[[HOTEL#CLEANLINESS, positive], [LOCATION#GENE...","[HOTEL#CLEANLINESS, LOCATION#GENERAL, SERVICE#...","[positive, positive, positive, positive, posit...",-0.5,1.0,1.0,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5


In [13]:
for i in range(len(df_list)):
    df_list[i].to_csv(str('csv_data/{}.csv'.format(files[i][9:len(files[i])-4])), index=False)